In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('titanic.csv')

# 1. Name 전처리

In [3]:
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.')

# 2. Age 전처리

In [4]:
title_list = df['Title'].unique()

title_mapping = dict()
for key, index in enumerate(title_list):
    if key >= 3:
        title_mapping[index] = 3
    else:
        title_mapping[index] = key
        
df['Title2'] = df['Title'].map(title_mapping)
df['Age'] = df['Age'].fillna(df.groupby('Title2')['Age'].transform("median"))

df.loc[df['Age'] <= 16, 'Age2'] = 0
df.loc[(16 < df['Age']) & (df['Age'] <= 26), 'Age2'] = 1
df.loc[(26 < df['Age']) & (df['Age'] <= 36), 'Age2'] = 2
df.loc[(36 < df['Age']) & (df['Age'] <= 62), 'Age2'] = 3
df.loc[df['Age'] > 62, 'Age2'] = 4

# 3. Sex 전처리

In [5]:
sex_mapping = {'male':0, 'female':1}
df['Sex2'] = df['Sex'].map(sex_mapping)

# 4. Embarked 전처리

In [6]:
df['Embarked'].fillna('S', inplace=True)

embarked_mapping = {'S':0, 'C':1, 'Q':2}
df['Embarked2'] = df['Embarked'].map(embarked_mapping)

# 5. Fare 전처리

In [7]:
df.loc[df['Fare'] <= 17, 'Fare2'] = 0
df.loc[(17 < df['Fare']) & (df['Fare'] <= 30), 'Fare2'] = 1
df.loc[(30 < df['Fare']) & (df['Fare'] <= 100), 'Fare2'] = 2
df.loc[df['Fare'] > 100, 'Fare2'] = 3

# 6. Cabin 전처리

In [8]:
df['Cabin2'] = df['Cabin'].str[0]

cabin_mapping = {'A':0, 'B':0.4, 'C':0.8, 'D':1.2, 'E':1.6, 'F':2, 'G':2.4, 'T':2.8}
df['Cabin2'] = df['Cabin2'].map(cabin_mapping)

df['Cabin2'].fillna(df.groupby('Pclass')['Cabin2'].transform('median'), inplace=True)

# 7. FamilySize 추가

In [9]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

family_mapping = {}
for value in range(1, df['FamilySize'].max()+1):
    family_mapping[value] = round((value-1) * 0.4, 2)

df['FamilySize'] = df['FamilySize'].map(family_mapping)

# 최종 결과

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    object 
 12  Title        891 non-null    object 
 13  Title2       891 non-null    int64  
 14  Age2         891 non-null    float64
 15  Sex2         891 non-null    int64  
 16  Embarked2    891 non-null    int64  
 17  Fare2        891 non-null    float64
 18  Cabin2       891 non-null    float64
 19  FamilySi

In [11]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Title2,Age2,Sex2,Embarked2,Fare2,Cabin2,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,1.0,0,0,0.0,2.0,0.4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,1,3.0,1,1,2.0,0.8,0.4
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,2,1.0,1,0,0.0,2.0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,1,2.0,1,0,2.0,0.8,0.4
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,0,2.0,0,0,0.0,2.0,0.0


In [15]:
features_drop = ['PassengerId', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'Title2']

In [16]:
df.drop(features_drop, axis=1)

,Survived,Pclass,Age2,Sex2,Embarked2,Fare2,Cabin2,FamilySize
0,0,3,1.0,0,0,0.0,2.0,0.4
1,1,1,3.0,1,1,2.0,0.8,0.4
2,1,3,1.0,1,0,0.0,2.0,0.0
3,1,1,2.0,1,0,2.0,0.8,0.4
4,0,3,2.0,0,0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...
886,0,2,2.0,0,0,0.0,1.8,0.0
887,1,1,1.0,1,0,1.0,0.4,0.0
888,0,3,1.0,1,0,1.0,2.0,1.2
889,1,1,1.0,0,1,1.0,0.8,0.0
